# Reviews
## keep only last 28 days, all text and images domain removed

In [87]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col , from_unixtime
#from pyspark.sql.functions import to_date, date_format, col, when
from datetime import datetime


# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read JSON with PySpark") \
    .getOrCreate()

In [88]:
# Read JSON file into DataFrame

reviews_schema = StructType([
    StructField("reviewerID", StringType(), False),
    StructField("asin", StringType(), False),
    StructField("overall", DoubleType(), False),
    StructField("reviewerName", StringType(), False),   
    StructField("reviewText", StringType(), False),   
    StructField("summary", StringType(), False), 
    StructField("unixReviewTime", StringType(), False)


    
    
])


reviews = spark.read.json("item_dedup.json", schema=reviews_schema)

In [89]:
# Display DataFrame schema
reviews.printSchema()
print((reviews.count(), len(reviews.columns)))


root
 |-- reviewerID: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: string (nullable = true)

(83057018, 7)


In [90]:
# Display first few rows of DataFrame
#reviews = reviews.withColumn("date_column", from_unixtime("unixReviewTime"))
#f = reviews.orderBy(reviews["date_column"].desc())
#f.show()

In [91]:
reviews_recent = reviews.filter(col("unixReviewTime") >= 1401577200)  # Unix timestamp for june 1st, 2014

print((reviews_recent.count(), len(reviews_recent.columns)))
# Display first few rows of DataFrame
reviews_recent.show()

(4958839, 7)
+--------------+----------+-------+--------------------+--------------------+--------------------+--------------+
|    reviewerID|      asin|overall|        reviewerName|          reviewText|             summary|unixReviewTime|
+--------------+----------+-------+--------------------+--------------------+--------------------+--------------+
|A1MCJONUQ78L9T|0000031887|    5.0|     Amazon Customer|Purchased it for ...|My daughter loved...|    1402876800|
|A28E4JIUJA10DK|0000031887|    5.0|               doris|This is perfect m...|Girl Ballet TuTu Red|    1402272000|
|A3KA7Y9BUFBLZG|0000031887|    5.0|              hariet|A beautiful yello...|          great tutu|    1401926400|
|A186E2SU6URFZI|0000031887|    1.0|                 Jae|Very cute however...|   Wouldnt buy again|    1402531200|
| A9A1P7C3G1V9H|0000031909|    2.0|      michelle smith|Cheap fabric, it ...|           Two Stars|    1404950400|
|A3TBJ4L4RGPIGE|0000032034|    5.0|          D. Simpson|As described...th..

# export back to json then load onto S3, then into Snowflake

In [92]:
reviews_recent = reviews_recent.repartition(1)
reviews_recent.write.mode("overwrite").json("june2014")

In [93]:
print(datetime.now())

2024-04-20 18:33:56.591807


In [94]:
spark.stop()